In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("."))

['post_data.csv', 'post_data_full.csv', '.DS_Store', 'data_collect.py', 'Applying Kaggle Spam Detection Model to Our Data.ipynb', 'README.md', '.ipynb_checkpoints', '.git', 'post_data_source.csv']


In [2]:
data = pd.read_csv("post_data_full.csv")

In [3]:
#Drop unnecessary columns 
data = data.drop(columns=['url', 'subreddit', 'score', 'permalink','Source'])

In [4]:
#show
data.head()

,class,title
0,1,COVID-19: Cases of vaccine-resistant variant m...
1,1,Should A COVID Vaccine Be Part Of The Back-To-...
2,1,UK hits 30 million first vaccine doses - 57% o...
3,1,EU medical regulator says AstraZeneca COVID va...
4,1,Moderna begins testing next-generation coronav...


In [5]:
#We choose 1 row.And we throw punctuation
import re
nlp_data = str(data.iloc[2,:])
nlp_data = re.sub("[^a-zA-Z]"," ",nlp_data)

In [6]:
nlp_data = nlp_data.lower()

In [7]:
import nltk as nlp
nlp_data = nlp.word_tokenize(nlp_data)
#nlp_data = nlp_data.split() or we can do so

In [8]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/david/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
#we have to find word root
lemma = nlp.WordNetLemmatizer()
nlp_data = [lemma.lemmatize(word) for word in nlp_data]

In [10]:
nlp_data = " ".join(nlp_data)

In [11]:
import nltk as nlp
import re
description_list = []
for description in data["title"]:
    description = re.sub("[^a-zA-Z]"," ",description)
    description = description.lower()   # buyuk harftan kucuk harfe cevirme
    description = nlp.word_tokenize(description)
    #description = [ word for word in description if not word in set(stopwords.words("english"))]
    lemma = nlp.WordNetLemmatizer()
    description = [ lemma.lemmatize(word) for word in description]
    description = " ".join(description)
    description_list.append(description) #we hide all word one section

In [12]:
#We make bag of word it is including number of all word's info
from sklearn.feature_extraction.text import CountVectorizer 
max_features = 3000 #We use the most common word
count_vectorizer = CountVectorizer(max_features = max_features, stop_words = "english")
sparce_matrix = count_vectorizer.fit_transform(description_list).toarray()
print("the most using {} words: {}".format(max_features,count_vectorizer.get_feature_names()))

the most using 3000 words: ['aaa', 'abandon', 'abandoned', 'abdul', 'ability', 'able', 'abound', 'abroad', 'absolutely', 'absurd', 'abu', 'academy', 'accelerate', 'accelerated', 'acceleration', 'acceptance', 'access', 'accessible', 'accidentally', 'accidently', 'according', 'account', 'accused', 'accuses', 'achieve', 'achieved', 'achieving', 'achilles', 'acid', 'acquire', 'act', 'action', 'active', 'activist', 'activity', 'actor', 'actually', 'ad', 'adaptive', 'add', 'added', 'adding', 'addition', 'additional', 'address', 'adenovirus', 'adjust', 'adjusts', 'adjuvant', 'administer', 'administered', 'administering', 'administers', 'administration', 'administrator', 'admits', 'adolescent', 'adopts', 'adriano', 'adult', 'advance', 'advantage', 'adverse', 'advice', 'advise', 'advised', 'adviser', 'advisor', 'advisory', 'affair', 'affect', 'affecting', 'affiliation', 'affluent', 'afford', 'affordable', 'afghanistan', 'africa', 'african', 'age', 'aged', 'agency', 'aggressive', 'ago', 'agree',

In [13]:
#We separate our data is train and test
y = data.iloc[:,0].values   # male or female classes
x = sparce_matrix
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.1, random_state = 42)

In [14]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(x_train,y_train)
print("the accuracy of our model: {}".format(nb.score(x_test,y_test)))

the accuracy of our model: 0.7307692307692307


In [15]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter = 200)
lr.fit(x_train,y_train)
print("our accuracy is: {}".format(lr.score(x_test,y_test)))

our accuracy is: 0.8538461538461538


In [16]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train,y_train)
#print('Prediction: {}'.format(prediction))
print('With KNN (K=3) accuracy is: ',knn.score(x_test,y_test))

With KNN (K=3) accuracy is:  0.8346153846153846


In [27]:
#!pip install keras
!pip install tensorflow

In [24]:
x_train = x_train.reshape(2335,3000,1)
x_test = x_test.reshape(260,3000,1)

In [25]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics=["accuracy"])

# Fitting the RNN to the Training set
regressor.fit(x_test, y_test, epochs = 3, batch_size = 32)

Epoch 1/3
9/9 [==============================] - 35s 3s/step - loss: 0.7269 - accuracy: 0.1699
Epoch 2/3
9/9 [==============================] - 30s 3s/step - loss: 0.2609 - accuracy: 0.8240
Epoch 3/3
9/9 [==============================] - 30s 3s/step - loss: 0.1491 - accuracy: 0.8579


In [18]:
import tensorflow as tf
print(tf.__version__)

2.4.1
